OLD NOTEBOOK: SEE osm_pbf_power_data_extractor.py which does everything.

In [2]:
import os, sys, time

sys.path.append("../scripts")
from osm_data_config import AFRICA_CC

import requests
import shutil

# https://gitlab.com/dlr-ve-esy/esy-osmfilter/-/tree/master/
from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter, Node, Way, Relation
from esy.osmfilter import export_geojson
from esy.osmfilter import osm_info as osm_info
from esy.osmfilter import osm_pickle as osm_pickle
from contextlib import contextmanager

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
import geoplot
import matplotlib.pyplot as plt

# import logging
# logging.basicConfig()
# logger=logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# logger.setLevel(logging.WARNING)

In [2]:
# Redundant code to create AFRICA_CC

# from osm_data_config import CC
# geofabrik_regions = ["algeria","angola","benin","botswana","burkina_faso","burundi","cameroon","canary_islands","cape_verde","central_african_republic","chad","comores","congo_brazzaville","congo_democratic_republic","djibouti","egypt","equatorial_guinea","eritrea","ethiopia","gabon","ghana","guinea_bissau","guinea","ivory_coast","kenya","lesotho","liberia","libya","madagascar","malawi","mali","mauritania","mauritius","morocco","mozambique","namibia","niger","nigeria","rwanda","saint_helena_ascension_and_tristan_da_cunha","sao_tome_and_principe","senegal_and_gambia","seychelles","sierra_leone","somalia","south_africa_and_lesotho","south_africa","south_sudan","sudan","swaziland","tanzania","togo","tunisia","uganda","zambia","zimbabwe"]

# for region in geofabrik_regions:
#     try :
#         print('"'+list(CC.keys())[list(CC.values()).index(region.replace('_',' ').upper())]+'":"'+region+'"')
#     except:
#         print(region)
#         pass

## Just Substations (v1) code

In [11]:
# country_code = 'ZW'
# update = False # prevents re-download and filteration of file


def download_and_filter(country_code, update=False):
    country_name = AFRICA_CC[country_code]
    element_file_exists = False
    # json file for the Data dictionary
    # json file for the Elements dictionary is automatically written to 'data/osm/Elements'+filename)
    JSON_outputfile = os.path.join(
        os.getcwd(), "data", "osm", country_name + "_substations.json"
    )

    if os.path.exists(JSON_outputfile):
        element_file_exists = True

    geofabrik_filename = f"{country_name}-latest.osm.pbf"  # Filename for geofabrik
    geofabrik_url = f"https://download.geofabrik.de/africa/{geofabrik_filename}"  # https://download.geofabrik.de/africa/nigeria-latest.osm.pbf

    PBF_inputfile = os.path.join(
        os.getcwd(), "data", "osm", "pbf", geofabrik_filename
    )  # Input filepath

    if not os.path.exists(PBF_inputfile) or update:
        print(f"{geofabrik_filename} does not exist, downloading to {PBF_inputfile}")
        os.makedirs(
            os.path.dirname(PBF_inputfile), exist_ok=True
        )  # create data/osm directory
        with requests.get(geofabrik_url, stream=True) as r:
            with open(PBF_inputfile, "wb") as f:
                shutil.copyfileobj(r.raw, f)

    # see https://dlr-ve-esy.gitlab.io/esy-osmfilter/filter.html for filter structures

    prefilter = {
        Node: {"power": ["substation"]},
        Way: {"power": ["substation"]},
        Relation: {"power": ["substation"]},
    }
    blackfilter = [
        ("pipeline", "substation"),
    ]  # HACKY: due to esy.osmfilter
    whitefilter = [
        [
            ("power", "substation"),
        ],
    ]

    elementname = f"{country_code}_substations"

    if update is False and element_file_exists is True:
        create_elements = False  # Do not create elements again
        new_prefilter_data = False  # Do not pre-filter data again

        ###Hacking esy.osmfilter code
        Data = osm_info.ReadJason(JSON_outputfile, verbose="no")
        DataDict = {"Data": Data}
        osm_pickle.picklesave(
            DataDict,
            os.path.realpath(
                os.path.join(os.getcwd(), os.path.dirname(JSON_outputfile))
            ),
        )

        # print("Loading Pickle")
    else:
        create_elements = True
        new_prefilter_data = True
        print("Creating  New Elements")

    return run_filter(
        elementname,
        PBF_inputfile,
        JSON_outputfile,
        prefilter,
        whitefilter,
        blackfilter,
        NewPreFilterData=new_prefilter_data,
        CreateElements=create_elements,
        LoadElements=True,
        verbose=False,
        multiprocess=True,
    )

In [5]:
df_all = pd.DataFrame()
for country_code in AFRICA_CC.keys():
    [Data, Elements] = download_and_filter(country_code)
    elementname = f"{country_code}_substations"
    df_way = pd.json_normalize(Elements[elementname]["Way"].values())
    df_node = pd.json_normalize(Elements[elementname]["Node"].values())

    convert_ways_nodes(df_way)
    # Add Type Column
    df_node["Type"] = "Node"
    df_way["Type"] = "Way"

    df_combined = pd.concat([df_node, df_way], axis=0)
    # Add Country Column
    df_combined["Country"] = AFRICA_CC[country_code]

    df_all = pd.concat([df_all, df_combined])


df_all.dropna(
    subset=["tags.voltage"], inplace=True
)  # Drop any substations with Voltage = N/A
df_all.dropna(
    thresh=len(df_all) * 0.25, axis=1, how="all", inplace=True
)  # Drop Columns with 75% values as N/A
# display(df_all)


# Generate Files
outputfile_partial = os.path.join(os.getcwd(), "data", "africa_all" + "_substations.")

df_all.to_csv(outputfile_partial + "csv")  # Generate CSV

gdf = convert_pd_to_gdf(df_all)
gdf.to_file(outputfile_partial + "geojson", driver="GeoJSON")  # Generate GeoJson

NameError: name 'download_and_filter' is not defined

In [2]:
# Convert Ways to Nodes
def convert_ways_nodes(df_way, Data):
    lonlat_column = []
    for ref in df_way["refs"]:
        lonlats = []
        for r in ref:
            lonlat = Data["Node"][str(r)]["lonlat"]
            lonlats.append(lonlat)
        lonlats = np.array(lonlats)
        lonlat = np.mean(lonlats, axis=0)  # Hacky Apporx Centroid
        lonlat_column.append(lonlat)
    df_way.drop("refs", axis=1, inplace=True)
    df_way.insert(1, "lonlat", lonlat_column)


# polygon_column=[]
# for poly_list in lonlat_column:
#     for p in poly_list:
#         print(p)

#     print("p")

In [3]:
def convert_pd_to_gdf(df_way):
    gdf = gpd.GeoDataFrame(df_way, geometry=[Point(x, y) for x, y in df_way.lonlat])
    gdf.drop(columns=["lonlat"], inplace=True)
    return gdf

In [4]:
def download_pbf(country_code, update):
    country_name = AFRICA_CC[country_code]
    geofabrik_filename = f"{country_name}-latest.osm.pbf"  # Filename for geofabrik
    geofabrik_url = f"https://download.geofabrik.de/africa/{geofabrik_filename}"  # https://download.geofabrik.de/africa/nigeria-latest.osm.pbf
    PBF_inputfile = os.path.join(
        os.getcwd(), "data", "osm", "pbf", geofabrik_filename
    )  # Input filepath

    if not os.path.exists(PBF_inputfile) or update:
        print(f"{geofabrik_filename} does not exist, downloading to {PBF_inputfile}")
        os.makedirs(
            os.path.dirname(PBF_inputfile), exist_ok=True
        )  # create data/osm directory
        with requests.get(geofabrik_url, stream=True) as r:
            with open(PBF_inputfile, "wb") as f:
                shutil.copyfileobj(r.raw, f)

    return PBF_inputfile

## Substation and Lines (v2) code

In [5]:
def download_and_filter(country_code, update=False):
    PBF_inputfile = download_pbf(country_code, update)

    filter_file_exists = False
    JSON_outputfile = os.path.join(
        os.getcwd(), "data", "osm", country_code + "_power.json"
    )

    if os.path.exists(JSON_outputfile):
        filter_file_exists = True

    if update is False and filter_file_exists is True:
        create_elements = False  # Do not create elements again
        new_prefilter_data = False  # Do not pre-filter data again
        ###Hacking esy.osmfilter code to re-create Data.pickle
        Data = osm_info.ReadJason(JSON_outputfile, verbose="no")
        DataDict = {"Data": Data}
        osm_pickle.picklesave(
            DataDict,
            os.path.realpath(
                os.path.join(os.getcwd(), os.path.dirname(JSON_outputfile))
            ),
        )
        print("Loading Pickle")
    else:
        create_elements = True
        new_prefilter_data = True
        print("Creating  New Elements")

    prefilter = {
        Node: {"power": ["substation", "line"]},
        Way: {"power": ["substation", "line"]},
        Relation: {"power": ["substation", "line"]},
    }
    blackfilter = [
        ("pipeline", "substation"),
    ]  # HACKY: due to esy.osmfilter

    for feature in ["substation", "line"]:
        whitefilter = [
            [
                ("power", feature),
            ],
        ]
        elementname = f"{country_code}_{feature}s"

        feature_data = run_filter(
            elementname,
            PBF_inputfile,
            JSON_outputfile,
            prefilter,
            whitefilter,
            blackfilter,
            NewPreFilterData=new_prefilter_data,
            CreateElements=create_elements,
            LoadElements=True,
            verbose=False,
            multiprocess=True,
        )

        if feature == "substation":
            substation_data = feature_data
        if feature == "line":
            line_data = feature_data

    return (substation_data, line_data)

In [6]:
def convert_filtered_data_to_dfs(feature_data, feature):
    [Data, Elements] = feature_data
    elementname = f"{country_code}_{feature}s"
    df_way = pd.json_normalize(Elements[elementname]["Way"].values())
    df_node = pd.json_normalize(Elements[elementname]["Node"].values())
    return (df_node, df_way, Data)

In [7]:
def process_substation_data(substation_data):
    df_node, df_way, Data = convert_filtered_data_to_dfs(substation_data, "substation")
    convert_ways_nodes(df_way, Data)
    # Add Type Column
    df_node["Type"] = "Node"
    df_way["Type"] = "Way"

    df_combined = pd.concat([df_node, df_way], axis=0)
    # Add Country Column
    df_combined["Country"] = AFRICA_CC[country_code]

    return df_combined

In [8]:
def process_line_data(line_data):
    df_node, df_way, Data = convert_filtered_data_to_dfs(line_data, "line")
    convert_ways_lines(df_way, Data)
    # Add Type Column
    df_way["Type"] = "Way"

    # Add Country Column
    df_way["Country"] = AFRICA_CC[country_code]
    return df_way

In [9]:
def convert_ways_lines(df_way, Data):
    lonlat_column = []
    for ref in df_way["refs"]:  # goes through each row in df_way['refs']
        lonlats = []
        for (
            r
        ) in (
            ref
        ):  # picks each element in ref & replaces ID by coordinate tuple (A multiline consist of several points)
            lonlat = Data["Node"][str(r)][
                "lonlat"
            ]  # "r" is the ID in Data["Node"], ["lonlat"] a list of [x1,y1] (coordinates)
            lonlat = tuple(lonlat)
            lonlats.append(lonlat)  # a list with tuples
        lonlat_column.append(lonlats)  # adding a new list of tuples every row
    df_way.drop("refs", axis=1, inplace=True)
    df_way.insert(1, "lonlat", lonlat_column)

In [10]:
def convert_pd_to_gdf_lines(df_way):
    df_way["geometry"] = df_way["lonlat"].apply(lambda x: LineString(x))
    gdf = gpd.GeoDataFrame(df_way, geometry="geometry", crs="EPSG:4326")
    gdf.drop(columns=["lonlat"], inplace=True)
    ##voltage filter
    # gdf.rename(columns = {'tags.voltage':"voltage_V"}, inplace = True)
    # ## some transmission lines carry multiple voltages, having voltage_V = 10000;20000  (two lines)
    # # The following code keeps only the first information before the semicolon..
    # # Needs to be corrected in future, creating two lines with the same bus ID.
    # gdf['voltage_V'] = gdf['voltage_V'].str.split(';').str[0]
    # gdf['voltage_V'] = gdf['voltage_V'].astype(int)
    # gdf = gdf[gdf.voltage_V > 10000]
    return gdf

In [12]:
# test_CC = {"DZ": "algeria", "EG": "egypt", "NG": "nigeria"}
df_all_substations = pd.DataFrame()
df_all_lines = pd.DataFrame()
for country_code in AFRICA_CC.keys():
    substation_data, line_data = download_and_filter(country_code)
    for feature in ["line"]:
        if feature == "substation":
            df_substation = process_substation_data(substation_data)
            df_all_substations = pd.concat([df_all_substations, df_substation])
        if feature == "line":
            df_line = process_line_data(line_data)
            df_all_lines = pd.concat([df_all_lines, df_line])


# df_all_lines.dropna(subset=['tags.voltage'], inplace = True) # Drop any substations with Voltage = N/A
df_all_lines.dropna(
    thresh=len(df_all_lines) * 0.25, axis=1, how="all", inplace=True
)  # Drop Columns with 75% values as N/A
# display(df_all_lines)


# Generate Files
outputfile_partial = os.path.join(os.getcwd(), "data", "africa_all" + "_lines.")

df_all_lines.to_csv(outputfile_partial + "csv")  # Generate CSV

gdf = convert_pd_to_gdf_lines(df_all_lines)
gdf.to_file(outputfile_partial + "geojson", driver="GeoJSON")  # Generate GeoJson

Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle
Loading Pickle


In [12]:
df_all = pd.DataFrame()

In [47]:
# country_code = 'MA'
# [Data,Elements]=download_and_filter(country_code)
# elementname=f'{country_code}_substations'
# df_way = pd.json_normalize(Elements[elementname]["Way"].values())
# df_node = pd.json_normalize(Elements[elementname]["Node"].values())


# try :
#     df_way.dropna(subset=['tags.voltage'], inplace = True) # Drop any substations with Voltage = N/A
#     df_node.dropna(subset=['tags.voltage'], inplace = True) # Drop any substations with Voltage = N/A
#     df_way.dropna(thresh=len(df_way)*0.25, axis=1, how='all', inplace = True) #Drop Columns with 75% values as N/A
#     df_node.dropna(thresh=len(df_node)*0.25, axis=1, how='all', inplace = True) #Drop Columns with 75% values as N/A
# except:
#     print('Problem with' + country_code)
#     pass

# # Add Type Column
# df_node['Type'] = 'Node'
# df_way['Type'] = 'Way'
# display(df_node)
# convert_ways_nodes(df_way)
# display(df_way)

# df_combined = pd.concat([df_node, df_way], axis = 0)
# df_combined.dropna(thresh=len(df_combined)*0.25, axis=1, how='all', inplace = True) #Drop Columns with 75% values as N/A
# display(df_combined)
# gdf = convert_pd_to_gdf(df_combined)
# display(gdf)